In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys; sys.path.append('../')
%matplotlib inline
import numpy as np
import sys,os
import pickle, gzip
import pandas as pd
import matplotlib,matplotlib.pyplot as plt
import utils
import encoders
import pytorch_models
import pandas as pd
import sklearn, sklearn.model_selection, sklearn.neighbors
import sklearn.linear_model, sklearn.ensemble
import gzip
import utils
import encoders
import collections
from mlp import MLP_train, MLP

In [4]:
import torch
import torch.optim
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import sklearn, sklearn.metrics

In [25]:
results = pd.read_csv("results-bal2.csv")

In [26]:
results2 = results.copy().fillna("")
results2 = results2[results2.label_type == "btype"]
#results2 = results2[results2.num_examples > 50]
results2.num_examples = results2.num_examples.astype(int)
results2.groupby(["label_type", "num_examples", "model","encode_method"]).agg({
                  'bacc':['mean','std','count']}).round(2)

bacc            
                                                       mean   std count
label_type num_examples model       encode_method                      
btype      48           conv-basic                     0.36  0.07    10
                        conv-resnet                    0.31  0.07    10
                        knn                            0.33  0.07    10
                                    biosppy_mean_beat  0.29  0.10    10
                                    convautoencoder    0.34  0.12    10
                                    fft                0.33  0.09    10
                                    pca_50             0.35  0.10    10
                                    periodogram        0.32  0.08    10
                        mlp                            0.34  0.11    10
           120          conv-basic                     0.40  0.08    10
                        conv-resnet                    0.34  0.04    10
                        knn                            0.37  0.04    10
                                    biosppy_mean_beat  0.33  0.08    10
                                    convautoencoder    0.42  0.06    10
                                    fft                0.39  0.07    10
                                    pca_50             0.38  0.08    10
                                    periodogram        0.36  0.06    10
                        mlp                            0.39  0.08    10
           1200         conv-basic                     0.69  0.03    10
                        conv-resnet                    0.72  0.04    10
                        knn                            0.53  0.03    10
                                    biosppy_mean_beat  0.37  0.02    10
                                    convautoencoder    0.58  0.03    10
                                    fft                0.49  0.02    10
                                    pca_50             0.59  0.03    10
                                    periodogram        0.42  0.02    10
                        mlp                            0.54  0.02    10
           12000        conv-basic                     0.80  0.01    10
                        conv-resnet                    0.85  0.03    10
                        knn                            0.65  0.01    10
                                    biosppy_mean_beat  0.40  0.01    10
                                    convautoencoder    0.68  0.01    10
                                    fft                0.57  0.01    10
                                    pca_50             0.67  0.01    10
                                    periodogram        0.50  0.01    10
                        mlp                            0.69  0.01    10

In [16]:
results2 = results.copy().fillna("None")
del results2["seed"]
a = results2.groupby(["label_type", "num_examples", "model","encode_method"]).mean(["bacc"])
a

bacc
label_type num_examples model       encode_method            
btype      48.0         conv-basic  None             0.351852
                        conv-resnet None             0.296296
                        knn         None             0.324074
                                    convautoencoder  0.333333
                                    fft              0.342593
                                    pca_50           0.379630
                        mlp         None             0.351852
           120.0        conv-basic  None             0.411111
                        conv-resnet None             0.333333
                        knn         None             0.362963
                                    convautoencoder  0.418519
                                    fft              0.392593
                                    pca_50           0.344444
                        mlp         None             0.355556
           1200.0       conv-basic  None             0.675556
                        conv-resnet None             0.719630
                        knn         None             0.525556
                                    convautoencoder  0.578519
                                    fft              0.498889
                                    pca_50           0.589630
                        mlp         None             0.539259
           12000.0      conv-basic  None             0.801630
                        conv-resnet None             0.861852
                        knn         None             0.639074
                                    convautoencoder  0.672741
                                    fft              0.567926
                                    pca_50           0.667333
                        mlp         None             0.688667
rtype      48.0         conv-basic  None             0.361111
                        conv-resnet None             0.333333
                        knn         None             0.324074
                                    convautoencoder  0.342593
                                    fft              0.324074
                                    pca_50           0.379630
                                    periodogram      0.333333
                        mlp         None             0.351852
           120.0        conv-basic  None             0.362963
                        conv-resnet None             0.355556
                        knn         None             0.337037
                                    convautoencoder  0.340741
                                    fft              0.403704
                                    pca_50           0.344444
                        mlp         None             0.400000
           1200.0       conv-basic  None             0.400741
                        conv-resnet None             0.513704
                        knn         None             0.382222
                                    convautoencoder  0.417037
                                    fft              0.476296
                                    pca_50           0.404815
                        mlp         None             0.398889
           12000.0      conv-basic  None             0.650778
                        conv-resnet None             0.879556
                        knn         None             0.446519
                                    convautoencoder  0.474778
                                    fft              0.611852
                                    pca_50           0.467074
                        mlp         None             0.515370